In [38]:
import sys
# 再帰回数上限変更
sys.setrecursionlimit(10**9)
import pypyjit
pypyjit.set_param('max_unroll_recursion=-1') # pypyの再帰高速化
from collections import defaultdict

num_dict = defaultdict(int)

# 関節点(Articulation Points)
def get_articulation_points(G, start=0):
    order = defaultdict(lambda: None)
    result = []; count = 0
    # 関節点を削除すると，何個の連結成分に分かれるか
    global num_dict
    def dfs(v, prev):
        nonlocal count
        r_min = order[v] = count # 到達時にラベル
        fcnt = 0; p_art = 0
        count += 1
        for w in G[v]:
            if w == prev:
                continue
            if order[w] is None:
                ret = dfs(w, v)
                # 子の頂点が到達できたのが、自身のラベル以上の頂点のみ
                # => 頂点vは関節点
                p_art += (order[v] <= ret)
                r_min = min(r_min, ret)
                fcnt += 1
            else:
                r_min = min(r_min, order[w])
        flag = (r_min == order[v] and len(G[v]) > 1)
        if (prev == -1 and fcnt > 1) or (prev != -1 and (p_art or flag)):
            # 頂点startの場合は、二箇所以上の子頂点を調べたら自身は関節点
            result.append(v)
            if prev == -1:
                num_dict[v] = fcnt
            else:
                num_dict[v] = p_art + 1
        return r_min
    dfs(start, -1)
    return result

# UnionFind
# n: 要素数 (0~n-1)
# parents: 各要素の親要素の番号を格納するリスト(要素が根の場合はそのグループの要素数を返す)
# find(x): 要素xが属するグループの根を返す
# union(x, y): 要素xが属するグループと要素yが属するグループとを併合する
# size(x): 要素xが属するグループのサイズ(要素数)を返す
# same(x, y): 要素x, yが同じグループに属するかどうかを返す
# members(x): 要素xが属するグループに属する要素をリストで返す
# roots(): 全ての根の要素をリストで返す
# group_count(): グループの数を返す
# all_group_members(): {根: そのグループに含まれる要素のリスト, ....}のdefaultdictを返す
# __str__(): print()での表示用

class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def size(self, x):
        return -self.parents[self.find(x)]

    def same(self, x, y):
        return self.find(x) == self.find(y)

    def members(self, x):
        root = self.find(x)
        return [i for i in range(self.n) if self.find(i) == root]

    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]

    def group_count(self):
        return len(self.roots())

    def all_group_members(self):
        group_members = defaultdict(list)
        for member in range(self.n):
            group_members[self.find(member)].append(member)
        return group_members

    def __str__(self):
        return '\n'.join(f'{r}: {m}' for r, m in self.all_group_members().items())
    
# 辺がない場合は-1，辺が一つしかない場合は+-0，関節点の場合は祖先への後退編の数
def main():
    H,W = map(int,input().split())
    mod = 998244353
    S = []
    for _ in range(H):
        S.append(input())
    uf = UnionFind(H*W)
    adj_dict = defaultdict(list)
    red_cnt = 0
    for i in range(H):
        for j in range(W):
            if S[i][j] == ".":
                red_cnt += 1
                continue
            if i+1 < H and S[i+1][j] == "#":
                uf.union(i*W+j,(i+1)*W+j)
                adj_dict[i*W+j].append((i+1)*W+j)
                adj_dict[(i+1)*W+j].append(i*W+j)
            if j+1 < W and S[i][j+1] == "#":
                uf.union(i*W+j,i*W+j+1)
                adj_dict[i*W+j].append(i*W+j+1)
                adj_dict[i*W+j+1].append(i*W+j)
    # 関節点の列挙
    art_points = set()
    for root in uf.roots():
        if uf.size(root) == 1 and S[root//W][root%W] == ".":
            continue
        art_points |= set(get_articulation_points(adj_dict, root))
    group = uf.group_count() - red_cnt
    ans_sum = group * (H*W-red_cnt)
    global num_dict
    for i in range(H):
        for j in range(W):
            if S[i][j] == ".":
                continue
            if adj_dict[i*W+j] == []:
                ans_sum -= 1
                continue
            if len(adj_dict[i*W+j]) == 1:
                continue
            if i*W+j in art_points:
                ans_sum += num_dict[i*W+j] - 1
                continue
    print(ans_sum * pow(H*W-red_cnt,-1,mod) % mod)

if __name__ == "__main__":
    main()

In [ ]:
from types import GeneratorType
import sys
sys.setrecursionlimit(10**9)
import pypyjit
pypyjit.set_param('max_unroll_recursion=-1')
from collections import defaultdict

def bootstrap(f, stack=[]):
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        to = f(*args, **kwargs)
        while True:
            if type(to) is GeneratorType:
                stack.append(to)
                to = next(to)
            else:
                stack.pop()
                if not stack:
                    break
                to = stack[-1].send(to)
        return to
    return wrappedfunc

num_dict = defaultdict(int)

# 関節点(Articulation Points)
def get_articulation_points(G, start=0):
    order = defaultdict(lambda: None)
    result = []; count = 0
    # 関節点を削除すると，何個の連結成分に分かれるか
    global num_dict
    @bootstrap
    def dfs(v, prev):
        nonlocal count
        r_min = order[v] = count # 到達時にラベル
        fcnt = 0; p_art = 0
        count += 1
        for w in G[v]:
            if w == prev:
                continue
            if order[w] is None:
                ret = yield dfs(w, v)
                # 子の頂点が到達できたのが、自身のラベル以上の頂点のみ
                # => 頂点vは関節点
                p_art += (order[v] <= ret)
                r_min = min(r_min, ret)
                fcnt += 1
            else:
                r_min = min(r_min, order[w])
        flag = (r_min == order[v] and len(G[v]) > 1)
        if (prev == -1 and fcnt > 1) or (prev != -1 and (p_art or flag)):
            # 頂点startの場合は、二箇所以上の子頂点を調べたら自身は関節点
            result.append(v)
            if prev == -1:
                num_dict[v] = fcnt
            else:
                num_dict[v] = p_art + 1
        yield r_min
    yield dfs(start, -1)
    return result

# UnionFind
# n: 要素数 (0~n-1)
# parents: 各要素の親要素の番号を格納するリスト(要素が根の場合はそのグループの要素数を返す)
# find(x): 要素xが属するグループの根を返す
# union(x, y): 要素xが属するグループと要素yが属するグループとを併合する
# size(x): 要素xが属するグループのサイズ(要素数)を返す
# same(x, y): 要素x, yが同じグループに属するかどうかを返す
# members(x): 要素xが属するグループに属する要素をリストで返す
# roots(): 全ての根の要素をリストで返す
# group_count(): グループの数を返す
# all_group_members(): {根: そのグループに含まれる要素のリスト, ....}のdefaultdictを返す
# __str__(): print()での表示用

class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return
        
        if self.parents[x] > self.parents[y]:
            x, y = y, x

        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def size(self, x):
        return -self.parents[self.find(x)]

    def same(self, x, y):
        return self.find(x) == self.find(y)

    def members(self, x):
        root = self.find(x)
        return [i for i in range(self.n) if self.find(i) == root]

    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]

    def group_count(self):
        return len(self.roots())

    def all_group_members(self):
        group_members = defaultdict(list)
        for member in range(self.n):
            group_members[self.find(member)].append(member)
        return group_members

    def __str__(self):
        return '\n'.join(f'{r}: {m}' for r, m in self.all_group_members().items())
    
# 辺がない場合は-1，辺が一つしかない場合は+-0，関節点の場合は祖先への後退編の数
def main():
    H,W = map(int,input().split())
    mod = 998244353
    S = []
    for _ in range(H):
        S.append(input())
    uf = UnionFind(H*W)
    adj_dict = defaultdict(list)
    red_cnt = 0
    for i in range(H):
        for j in range(W):
            if S[i][j] == ".":
                red_cnt += 1
                continue
            if i+1 < H and S[i+1][j] == "#":
                uf.union(i*W+j,(i+1)*W+j)
                adj_dict[i*W+j].append((i+1)*W+j)
                adj_dict[(i+1)*W+j].append(i*W+j)
            if j+1 < W and S[i][j+1] == "#":
                uf.union(i*W+j,i*W+j+1)
                adj_dict[i*W+j].append(i*W+j+1)
                adj_dict[i*W+j+1].append(i*W+j)
    # 関節点の列挙
    art_points = set()
    for root in uf.roots():
        if uf.size(root) == 1 and S[root//W][root%W] == ".":
            continue
        art_points |= set(get_articulation_points(adj_dict, root))
    group = uf.group_count() - red_cnt
    ans_sum = group * (H*W-red_cnt)
    global num_dict
    for i in range(H):
        for j in range(W):
            if S[i][j] == ".":
                continue
            if adj_dict[i*W+j] == []:
                ans_sum -= 1
                continue
            if len(adj_dict[i*W+j]) == 1:
                continue
            if i*W+j in art_points:
                ans_sum += num_dict[i*W+j] - 1
                continue
    print(ans_sum * pow(H*W-red_cnt,-1,mod) % mod)

if __name__ == "__main__":
    main()